In [149]:
#import dependencies
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import LSTM, Dense, InputLayer, Dropout
from keras.callbacks import EarlyStopping
from sklearn.metrics import accuracy_score

In [150]:
df = pd.read_csv('nbaDatabasePreProcessed2004-2024.csv')
df

,date,homeTeam,homeTeam_id,homeTeam_win%,homeTeam_wins,homeTeam_losses,homeTeam_points_total,homeTeam_points_q1,homeTeam_points_q2,homeTeam_points_q3,...,awayTeam_D_Reb,awayTeam_Total_Reb,awayTeam_Ast,awayTeam_Stl,awayTeam_Blk,awayTeam_TO,awayTeam_PF,winner,season,isPlayoffGame
0,2024-06-17,Boston Celtics,BOS,0.797980,79,20,106,28,39,19,...,28,35,18,4,4,13,20,Boston Celtics,2023-2024,True
1,2024-06-14,Dallas Mavericks,DAL,0.613861,62,39,122,34,27,31,...,27,31,18,2,5,13,19,Dallas Mavericks,2023-2024,True
2,2024-06-12,Dallas Mavericks,DAL,0.610000,61,39,99,31,20,19,...,30,36,26,4,6,9,19,Boston Celtics,2023-2024,True
3,2024-06-09,Boston Celtics,BOS,0.802083,77,19,105,25,29,29,...,34,43,21,5,3,15,17,Boston Celtics,2023-2024,True
4,2024-06-06,Boston Celtics,BOS,0.800000,76,19,107,37,26,23,...,33,43,9,8,1,11,16,Boston Celtics,2023-2024,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25726,2004-11-03,Cleveland Cavaliers,CLE,0.000000,0,0,104,21,19,27,...,36,47,24,9,5,13,28,Indiana Pacers,2004-2005,False
25727,2004-11-03,Boston Celtics,BOS,0.000000,0,0,95,22,31,30,...,27,41,20,9,2,16,26,Philadelphia 76ers,2004-2005,False
25728,2004-11-02,Los Angeles Lakers,LAL,0.000000,0,0,89,24,26,22,...,32,48,17,10,8,16,21,Los Angeles Lakers,2004-2005,False
25729,2004-11-02,Detroit Pistons,DET,0.000000,0,0,87,19,18,22,...,27,36,8,4,5,16,24,Detroit Pistons,2004-2005,False


In [151]:
#maps franchises to subjectID
subjectID_dict = {
    "ATL": 1,
    "BOS": 2,
    "BKN": 3,
    "CHA": 4,
    "CHI": 5,
    "CLE": 6,
    "DAL": 7,
    "DEN": 8,
    "DET": 9,
    "GSW": 10,
    "HOU": 11,
    "IND": 12,
    "LAC": 13,
    "LAL": 14,
    "MEM": 15,
    "MIA": 16,
    "MIL": 17,
    "MIN": 18,
    "NOP": 19,
    "NYK": 20,
    "OKC": 21,
    "ORL": 22,
    "PHI": 23,
    "PHX": 24,
    "POR": 25,
    "SAC": 26,
    "SAS": 27,
    "TOR": 28,
    "UTA": 29,
    "WAS": 30,
    "NOH": 19, #New Orleans Hornets is same franchise as NOP- New Orleans Pelicans
    "NJN": 3, #New Jersey Nets relocated to Brooklyn
    "SEA": 21, #Seattle Supersonics Relocated to OKC
    "NOK": 19, #New Orleans Hornets relocated to OKC 2005-2007 - Eventually became New Orleans Pelicans
    
}

In [152]:
#generate the X Y matrices
def df_to_X_y(df, dfWindow, window_size):
    X = []
    y = []
    #each team must have played window_size games before data can be extracted
    for index, row in df.iterrows():
        homeTeam_id = row['homeTeamSubject_id']
        awayTeam_id = row['awayTeamSubject_id']
        date = row['date']
        #limit dfWindow to include rows that occured before date of row we are on
        dfWindow_before_date = dfWindow[dfWindow['date']< date]
        homeTeam_occurrences = dfWindow_before_date[dfWindow_before_date['team_id'] == homeTeam_id].shape[0]
        awayTeam_occurrences = dfWindow_before_date[dfWindow_before_date['team_id'] == awayTeam_id].shape[0]
        if homeTeam_occurrences > window_size and awayTeam_occurrences > window_size:
            homeTeam_window = (dfWindow_before_date[dfWindow_before_date['team_id'] == homeTeam_id].sort_values(by='date', ascending=False).head(window_size).drop(columns=['date', 'team_id']))
            awayTeam_window = (dfWindow_before_date[dfWindow_before_date['team_id'] == awayTeam_id].sort_values(by='date', ascending=False).head(window_size).drop(columns=['date', 'team_id']))
            homeTeam_window = homeTeam_window.to_numpy()
            awayTeam_window = awayTeam_window.to_numpy()
            combined_window = np.hstack((homeTeam_window, awayTeam_window))
            X.append(combined_window)
            y.append(row['winner_binary'])
    
    return np.array(X), np.array(y)

In [153]:
#map the subject IDs to a number to handle franchise moves
df["homeTeamSubject_id"] = df["homeTeam_id"].map(subjectID_dict)
df["awayTeamSubject_id"] = df["awayTeam_id"].map(subjectID_dict)

cols = df.columns.tolist()
cols.insert(cols.index("homeTeam_id") + 1, cols.pop(cols.index("homeTeamSubject_id")))
cols.insert(cols.index("awayTeam_id") + 1, cols.pop(cols.index("awayTeamSubject_id")))
df = df[cols]

df

,date,homeTeam,homeTeam_id,homeTeamSubject_id,homeTeam_win%,homeTeam_wins,homeTeam_losses,homeTeam_points_total,homeTeam_points_q1,homeTeam_points_q2,...,awayTeam_D_Reb,awayTeam_Total_Reb,awayTeam_Ast,awayTeam_Stl,awayTeam_Blk,awayTeam_TO,awayTeam_PF,winner,season,isPlayoffGame
0,2024-06-17,Boston Celtics,BOS,2,0.797980,79,20,106,28,39,...,28,35,18,4,4,13,20,Boston Celtics,2023-2024,True
1,2024-06-14,Dallas Mavericks,DAL,7,0.613861,62,39,122,34,27,...,27,31,18,2,5,13,19,Dallas Mavericks,2023-2024,True
2,2024-06-12,Dallas Mavericks,DAL,7,0.610000,61,39,99,31,20,...,30,36,26,4,6,9,19,Boston Celtics,2023-2024,True
3,2024-06-09,Boston Celtics,BOS,2,0.802083,77,19,105,25,29,...,34,43,21,5,3,15,17,Boston Celtics,2023-2024,True
4,2024-06-06,Boston Celtics,BOS,2,0.800000,76,19,107,37,26,...,33,43,9,8,1,11,16,Boston Celtics,2023-2024,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25726,2004-11-03,Cleveland Cavaliers,CLE,6,0.000000,0,0,104,21,19,...,36,47,24,9,5,13,28,Indiana Pacers,2004-2005,False
25727,2004-11-03,Boston Celtics,BOS,2,0.000000,0,0,95,22,31,...,27,41,20,9,2,16,26,Philadelphia 76ers,2004-2005,False
25728,2004-11-02,Los Angeles Lakers,LAL,14,0.000000,0,0,89,24,26,...,32,48,17,10,8,16,21,Los Angeles Lakers,2004-2005,False
25729,2004-11-02,Detroit Pistons,DET,9,0.000000,0,0,87,19,18,...,27,36,8,4,5,16,24,Detroit Pistons,2004-2005,False


In [154]:
#one hot encode franchises - without dropping subject_id as it is needed for X_y function
df = pd.get_dummies(df, columns=['homeTeamSubject_id'], prefix='homeTeam', drop_first=False)
df = pd.get_dummies(df, columns=['awayTeamSubject_id'], prefix='awayTeam', drop_first=False)
df


,date,homeTeam,homeTeam_id,homeTeam_win%,homeTeam_wins,homeTeam_losses,homeTeam_points_total,homeTeam_points_q1,homeTeam_points_q2,homeTeam_points_q3,...,awayTeam_21,awayTeam_22,awayTeam_23,awayTeam_24,awayTeam_25,awayTeam_26,awayTeam_27,awayTeam_28,awayTeam_29,awayTeam_30
0,2024-06-17,Boston Celtics,BOS,0.797980,79,20,106,28,39,19,...,False,False,False,False,False,False,False,False,False,False
1,2024-06-14,Dallas Mavericks,DAL,0.613861,62,39,122,34,27,31,...,False,False,False,False,False,False,False,False,False,False
2,2024-06-12,Dallas Mavericks,DAL,0.610000,61,39,99,31,20,19,...,False,False,False,False,False,False,False,False,False,False
3,2024-06-09,Boston Celtics,BOS,0.802083,77,19,105,25,29,29,...,False,False,False,False,False,False,False,False,False,False
4,2024-06-06,Boston Celtics,BOS,0.800000,76,19,107,37,26,23,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25726,2004-11-03,Cleveland Cavaliers,CLE,0.000000,0,0,104,21,19,27,...,False,False,False,False,False,False,False,False,False,False
25727,2004-11-03,Boston Celtics,BOS,0.000000,0,0,95,22,31,30,...,False,False,True,False,False,False,False,False,False,False
25728,2004-11-02,Los Angeles Lakers,LAL,0.000000,0,0,89,24,26,22,...,False,False,False,False,False,False,False,False,False,False
25729,2004-11-02,Detroit Pistons,DET,0.000000,0,0,87,19,18,22,...,False,False,False,False,False,False,False,False,False,False


In [155]:
#bring back subject_id as it is needed for X_y function
df["homeTeamSubject_id"] = df["homeTeam_id"].map(subjectID_dict)
df["awayTeamSubject_id"] = df["awayTeam_id"].map(subjectID_dict)

cols = df.columns.tolist()
cols.insert(cols.index("homeTeam_id") + 1, cols.pop(cols.index("homeTeamSubject_id")))
cols.insert(cols.index("awayTeam_id") + 1, cols.pop(cols.index("awayTeamSubject_id")))
df = df[cols]

df

,date,homeTeam,homeTeam_id,homeTeamSubject_id,homeTeam_win%,homeTeam_wins,homeTeam_losses,homeTeam_points_total,homeTeam_points_q1,homeTeam_points_q2,...,awayTeam_21,awayTeam_22,awayTeam_23,awayTeam_24,awayTeam_25,awayTeam_26,awayTeam_27,awayTeam_28,awayTeam_29,awayTeam_30
0,2024-06-17,Boston Celtics,BOS,2,0.797980,79,20,106,28,39,...,False,False,False,False,False,False,False,False,False,False
1,2024-06-14,Dallas Mavericks,DAL,7,0.613861,62,39,122,34,27,...,False,False,False,False,False,False,False,False,False,False
2,2024-06-12,Dallas Mavericks,DAL,7,0.610000,61,39,99,31,20,...,False,False,False,False,False,False,False,False,False,False
3,2024-06-09,Boston Celtics,BOS,2,0.802083,77,19,105,25,29,...,False,False,False,False,False,False,False,False,False,False
4,2024-06-06,Boston Celtics,BOS,2,0.800000,76,19,107,37,26,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25726,2004-11-03,Cleveland Cavaliers,CLE,6,0.000000,0,0,104,21,19,...,False,False,False,False,False,False,False,False,False,False
25727,2004-11-03,Boston Celtics,BOS,2,0.000000,0,0,95,22,31,...,False,False,True,False,False,False,False,False,False,False
25728,2004-11-02,Los Angeles Lakers,LAL,14,0.000000,0,0,89,24,26,...,False,False,False,False,False,False,False,False,False,False
25729,2004-11-02,Detroit Pistons,DET,9,0.000000,0,0,87,19,18,...,False,False,False,False,False,False,False,False,False,False


In [156]:
#replace instances of true/false with 1,0
df = df.replace({True: 1, False: 0})
df

,date,homeTeam,homeTeam_id,homeTeamSubject_id,homeTeam_win%,homeTeam_wins,homeTeam_losses,homeTeam_points_total,homeTeam_points_q1,homeTeam_points_q2,...,awayTeam_21,awayTeam_22,awayTeam_23,awayTeam_24,awayTeam_25,awayTeam_26,awayTeam_27,awayTeam_28,awayTeam_29,awayTeam_30
0,2024-06-17,Boston Celtics,BOS,2,0.797980,79,20,106,28,39,...,0,0,0,0,0,0,0,0,0,0
1,2024-06-14,Dallas Mavericks,DAL,7,0.613861,62,39,122,34,27,...,0,0,0,0,0,0,0,0,0,0
2,2024-06-12,Dallas Mavericks,DAL,7,0.610000,61,39,99,31,20,...,0,0,0,0,0,0,0,0,0,0
3,2024-06-09,Boston Celtics,BOS,2,0.802083,77,19,105,25,29,...,0,0,0,0,0,0,0,0,0,0
4,2024-06-06,Boston Celtics,BOS,2,0.800000,76,19,107,37,26,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25726,2004-11-03,Cleveland Cavaliers,CLE,6,0.000000,0,0,104,21,19,...,0,0,0,0,0,0,0,0,0,0
25727,2004-11-03,Boston Celtics,BOS,2,0.000000,0,0,95,22,31,...,0,0,1,0,0,0,0,0,0,0
25728,2004-11-02,Los Angeles Lakers,LAL,14,0.000000,0,0,89,24,26,...,0,0,0,0,0,0,0,0,0,0
25729,2004-11-02,Detroit Pistons,DET,9,0.000000,0,0,87,19,18,...,0,0,0,0,0,0,0,0,0,0


In [157]:
#generate a binary winner column
df['winner_binary'] = (df['winner'] == df['awayTeam']).astype(int)
df

,date,homeTeam,homeTeam_id,homeTeamSubject_id,homeTeam_win%,homeTeam_wins,homeTeam_losses,homeTeam_points_total,homeTeam_points_q1,homeTeam_points_q2,...,awayTeam_22,awayTeam_23,awayTeam_24,awayTeam_25,awayTeam_26,awayTeam_27,awayTeam_28,awayTeam_29,awayTeam_30,winner_binary
0,2024-06-17,Boston Celtics,BOS,2,0.797980,79,20,106,28,39,...,0,0,0,0,0,0,0,0,0,0
1,2024-06-14,Dallas Mavericks,DAL,7,0.613861,62,39,122,34,27,...,0,0,0,0,0,0,0,0,0,0
2,2024-06-12,Dallas Mavericks,DAL,7,0.610000,61,39,99,31,20,...,0,0,0,0,0,0,0,0,0,1
3,2024-06-09,Boston Celtics,BOS,2,0.802083,77,19,105,25,29,...,0,0,0,0,0,0,0,0,0,0
4,2024-06-06,Boston Celtics,BOS,2,0.800000,76,19,107,37,26,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25726,2004-11-03,Cleveland Cavaliers,CLE,6,0.000000,0,0,104,21,19,...,0,0,0,0,0,0,0,0,0,1
25727,2004-11-03,Boston Celtics,BOS,2,0.000000,0,0,95,22,31,...,0,1,0,0,0,0,0,0,0,1
25728,2004-11-02,Los Angeles Lakers,LAL,14,0.000000,0,0,89,24,26,...,0,0,0,0,0,0,0,0,0,0
25729,2004-11-02,Detroit Pistons,DET,9,0.000000,0,0,87,19,18,...,0,0,0,0,0,0,0,0,0,0


In [139]:
print(list(df.columns))

['date', 'homeTeam', 'homeTeam_id', 'homeTeam_win%', 'homeTeam_wins', 'homeTeam_losses', 'homeTeam_points_total', 'homeTeam_points_q1', 'homeTeam_points_q2', 'homeTeam_points_q3', 'homeTeam_points_q4', 'homeTeam_points_1OT', 'homeTeam_points_2OT', 'homeTeam_points_3OT', 'homeTeam_points_4OT', 'homeTeam_FG', 'homeTeam_FG_made', 'homeTeam_3P', 'homeTeam_3P_made', 'homeTeam_O_Reb', 'homeTeam_D_Reb', 'homeTeam_Total_Reb', 'homeTeam_Ast', 'homeTeam_Stl', 'homeTeam_Blk', 'homeTeam_TO', 'homeTeam_PF', 'awayTeam', 'awayTeam_id', 'awayTeam_win%', 'awayTeam_wins', 'awayTeam_losses', 'awayTeam_points_total', 'awayTeam_points_q1', 'awayTeam_points_q2', 'awayTeam_points_q3', 'awayTeam_points_q4', 'awayTeam_points_1OT', 'awayTeam_points_2OT', 'awayTeam_points_3OT', 'awayTeam_points_4OT', 'awayTeam_FG', 'awayTeam_FG_made', 'awayTeam_3P', 'awayTeam_3P_made', 'awayTeam_O_Reb', 'awayTeam_D_Reb', 'awayTeam_Total_Reb', 'awayTeam_Ast', 'awayTeam_Stl', 'awayTeam_Blk', 'awayTeam_TO', 'awayTeam_PF', 'winner',

In [158]:
#populate dfIDtoStat - should be double the size of df
homedf = df[['date', 'homeTeamSubject_id', 'isPlayoffGame', 'homeTeam_win%', 'homeTeam_wins', 'homeTeam_losses', 'homeTeam_points_total', 'homeTeam_FG', 'homeTeam_FG_made', 'homeTeam_3P', 'homeTeam_3P_made', 'homeTeam_O_Reb', 'homeTeam_D_Reb', 'homeTeam_Total_Reb', 'homeTeam_Ast', 'homeTeam_Stl', 'homeTeam_Blk', 'homeTeam_TO', 'homeTeam_PF', 'homeTeam_1', 'homeTeam_2', 'homeTeam_3', 'homeTeam_4', 'homeTeam_5', 'homeTeam_6', 'homeTeam_7', 'homeTeam_8', 'homeTeam_9', 'homeTeam_10', 'homeTeam_11', 'homeTeam_12', 'homeTeam_13', 'homeTeam_14', 'homeTeam_15', 'homeTeam_16', 'homeTeam_17', 'homeTeam_18', 'homeTeam_19', 'homeTeam_20', 'homeTeam_21', 'homeTeam_22', 'homeTeam_23', 'homeTeam_24', 'homeTeam_25', 'homeTeam_26', 'homeTeam_27', 'homeTeam_28', 'homeTeam_29', 'homeTeam_30']]
homedf.columns = ['date', 'team_id', 'isPlayoffGame', 'team_win%', 'team_wins', 'team_losses', 'team_points_total', 'team_FG', 'team_FG_made', 'team_3P', 'team_3P_made', 'team_O_Reb', 'team_D_Reb', 'team_Total_Reb', 'team_Ast', 'team_Stl', 'team_Blk', 'team_TO', 'team_PF', 'team_1', 'team_2', 'team_3', 'team_4', 'team_5', 'team_6', 'team_7', 'team_8', 'team_9', 'team_10', 'team_11', 'team_12', 'team_13', 'team_14', 'team_15', 'team_16', 'team_17', 'team_18', 'team_19', 'team_20', 'team_21', 'team_22', 'team_23', 'team_24', 'team_25', 'team_26', 'team_27', 'team_28', 'team_29', 'team_30']
awaydf = df[['date', 'awayTeamSubject_id', 'isPlayoffGame', 'awayTeam_win%', 'awayTeam_wins', 'awayTeam_losses', 'awayTeam_points_total', 'awayTeam_FG', 'awayTeam_FG_made', 'awayTeam_3P', 'awayTeam_3P_made', 'awayTeam_O_Reb', 'awayTeam_D_Reb', 'awayTeam_Total_Reb', 'awayTeam_Ast', 'awayTeam_Stl', 'awayTeam_Blk', 'awayTeam_TO', 'awayTeam_PF', 'awayTeam_1', 'awayTeam_2', 'awayTeam_3', 'awayTeam_4', 'awayTeam_5', 'awayTeam_6', 'awayTeam_7', 'awayTeam_8', 'awayTeam_9', 'awayTeam_10', 'awayTeam_11', 'awayTeam_12', 'awayTeam_13', 'awayTeam_14', 'awayTeam_15', 'awayTeam_16', 'awayTeam_17', 'awayTeam_18', 'awayTeam_19', 'awayTeam_20', 'awayTeam_21', 'awayTeam_22', 'awayTeam_23', 'awayTeam_24', 'awayTeam_25', 'awayTeam_26', 'awayTeam_27', 'awayTeam_28', 'awayTeam_29', 'awayTeam_30']]
awaydf.columns = ['date', 'team_id', 'isPlayoffGame', 'team_win%', 'team_wins', 'team_losses', 'team_points_total', 'team_FG', 'team_FG_made', 'team_3P', 'team_3P_made', 'team_O_Reb', 'team_D_Reb', 'team_Total_Reb', 'team_Ast', 'team_Stl', 'team_Blk', 'team_TO', 'team_PF', 'team_1', 'team_2', 'team_3', 'team_4', 'team_5', 'team_6', 'team_7', 'team_8', 'team_9', 'team_10', 'team_11', 'team_12', 'team_13', 'team_14', 'team_15', 'team_16', 'team_17', 'team_18', 'team_19', 'team_20', 'team_21', 'team_22', 'team_23', 'team_24', 'team_25', 'team_26', 'team_27', 'team_28', 'team_29', 'team_30']

dfIDtoStat = pd.concat([homedf, awaydf], ignore_index=True)
dfIDtoStat

,date,team_id,isPlayoffGame,team_win%,team_wins,team_losses,team_points_total,team_FG,team_FG_made,team_3P,...,team_21,team_22,team_23,team_24,team_25,team_26,team_27,team_28,team_29,team_30
0,2024-06-17,2,1,0.797980,79,20,106,0.426966,38,0.333333,...,0,0,0,0,0,0,0,0,0,0
1,2024-06-14,7,1,0.613861,62,39,122,0.505495,46,0.405405,...,0,0,0,0,0,0,0,0,0,0
2,2024-06-12,7,1,0.610000,61,39,99,0.441860,38,0.360000,...,0,0,0,0,0,0,0,0,0,0
3,2024-06-09,2,1,0.802083,77,19,105,0.452381,38,0.256410,...,0,0,0,0,0,0,0,0,0,0
4,2024-06-06,2,1,0.800000,76,19,107,0.475610,39,0.380952,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51457,2004-11-03,12,0,0.000000,0,0,109,0.462366,43,0.235294,...,0,0,0,0,0,0,0,0,0,0
51458,2004-11-03,23,0,0.000000,0,0,98,0.444444,36,0.375000,...,0,0,1,0,0,0,0,0,0,0
51459,2004-11-02,8,0,0.000000,0,0,78,0.340909,30,0.250000,...,0,0,0,0,0,0,0,0,0,0
51460,2004-11-02,11,0,0.000000,0,0,79,0.394366,28,0.461538,...,0,0,0,0,0,0,0,0,0,0


In [159]:
#X.Shape is num data, window size, variables
X, y = df_to_X_y(df, dfIDtoStat, 5)
X.shape, y.shape

((25634, 5, 94), (25634,))

In [9]:
#80-10-10 split
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)
X_train.shape, y_train.shape, X_val.shape, y_val.shape, X_test.shape, y_test.shape

((17408, 5, 32), (17408,), (2176, 5, 32), (2176,), (2177, 5, 32), (2177,))

In [10]:
model = Sequential()
model.add(InputLayer((5, 32)))
model.add(LSTM(64, activation='tanh', return_sequences=True))
model.add(LSTM(32, activation='tanh'))
model.add(Dense(16, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 5, 64)             24832     
                                                                 
 lstm_1 (LSTM)               (None, 32)                12416     
                                                                 
 dense (Dense)               (None, 16)                528       
                                                                 
 dense_1 (Dense)             (None, 8)                 136       
                                                                 
 dense_2 (Dense)             (None, 1)                 9         
                                                                 
Total params: 37921 (148.13 KB)
Trainable params: 37921 (148.13 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [11]:
earlystopping = EarlyStopping(monitor='val_loss', patience=50, restore_best_weights=True)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [21]:
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=100, callbacks=[earlystopping])

Epoch 1/100
544/544 [==============================] - 3s 5ms/step - loss: 0.6233 - accuracy: 0.6484 - val_loss: 0.6179 - val_accuracy: 0.6682
Epoch 2/100
544/544 [==============================] - 3s 5ms/step - loss: 0.6244 - accuracy: 0.6477 - val_loss: 0.6141 - val_accuracy: 0.6664
Epoch 3/100
544/544 [==============================] - 3s 5ms/step - loss: 0.6239 - accuracy: 0.6500 - val_loss: 0.6255 - val_accuracy: 0.6581
Epoch 4/100
544/544 [==============================] - 3s 5ms/step - loss: 0.6221 - accuracy: 0.6530 - val_loss: 0.6141 - val_accuracy: 0.6714
Epoch 5/100
544/544 [==============================] - 3s 5ms/step - loss: 0.6216 - accuracy: 0.6543 - val_loss: 0.6234 - val_accuracy: 0.6549
Epoch 6/100
544/544 [==============================] - 3s 5ms/step - loss: 0.6226 - accuracy: 0.6514 - val_loss: 0.6157 - val_accuracy: 0.6590
Epoch 7/100
544/544 [==============================] - 3s 5ms/step - loss: 0.6212 - accuracy: 0.6545 - val_loss: 0.6115 - val_accuracy: 0.6682

KeyboardInterrupt: 